In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp ngram

In [ ]:
#export
from mlzero.segment import *
from mlzero.data import *
from typing import *
import bounter

# N-gram language models

## Generating n-grams

In [ ]:
#export
T = TypeVar('Token')

PAD = '_PAD_'

def ngrams(tokens: List[T], n: int, pad: T = PAD) -> List[Tuple[T, ...]]:
    """Returns list of ngrams from tokens adding padding as required
    
    Adds n-1 pad tokens at the start, and 1 to the end
    See https://skeptric.com/ngram-sentence-boundaries/
    """
    if pad is None:
        padded_tokens = tokens
    else:
        padded_tokens = [pad] * (n-1) + tokens + [pad]
    return list(zip(*[padded_tokens[i:] for i in range(n)]))

In [ ]:
text = "Sam I Am"

In [ ]:
tokens = tokenize_ascii(text)
tokens

['Sam', 'I', 'Am']

In [ ]:
assert ngrams(tokens, 1) == [
    ('Sam',),
    ('I',),
    ('Am',),
    (PAD,)]

In [ ]:
assert ngrams(tokens, 2) == [
    (PAD, 'Sam'),
    ('Sam', 'I',),
    ('I', 'Am',),
    ('Am', PAD,)]

Passing in a custom padding token

In [ ]:
pad = '_custom_pad_'
assert ngrams(tokens, 3, pad) == [
    (pad, pad, 'Sam'),
    (pad, 'Sam', 'I'),
    ('Sam', 'I', 'Am'),
    ('I', 'Am', pad)
] 

Passing `pad=None` removes the padding

In [ ]:
assert ngrams(tokens, 2, pad=None) == [
    ('Sam', 'I',),
    ('I', 'Am',),
] 

# Vocabulary Management

While it would be more efficient to construct a vocabulary on the fly, it's simpler to first create a vocabulary from a text and then build a model on it.

In [ ]:
#export
from tqdm.notebook import tqdm

In [ ]:
#export
def invert_vocab(v):
    return {v: i for i,v in enumerate(v)}

Todo: Non-desctructive tokenisation

In [ ]:
isinstance(range(0, 10), Generator)

False

In [ ]:
#export

OOV = '__OOV__'

OOV_IDX = 1
PAD_IDX = 0

class Vocab:
    def __init__(self, tokenize:Callable[[str], List[str]], tokens: List[str], oov: str=OOV, pad: str=PAD) -> None:
        self.tokenize = tokenize
        assert oov not in tokens
        assert pad not in tokens
        self.i2v = [pad, oov] + list(set(tokens))
        self.v2i = {v:i for i, v in enumerate(self.i2v)}
        self.size = len(self.i2v)
    
    def encode(self, text: str) -> List[int]:
        return [self.v2i.get(token, OOV_IDX) for token in self.tokenize(text)]
    
    def decode(self, tokens: List[int]) -> List[str]:
        return [self.i2v[token] for token in tokens]
    
    def __iter__(self) -> Generator[int, None, None]:
        """Iterate over the numerical tokens"""
        for idx in range(len(self.i2v)):
            yield idx
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return f'Vocab [{", ".join(self.i2v[2:6])}, ...] ({self.size} tokens)'
    
    def __eq__(self, other: Any) -> bool:
        if isinstance(other, Vocab):
            return self.i2v == other.i2v
        else:
            return False

In [ ]:
#export
def vocab_topn(corpora: List[str], tokenize: Callable[[str], List[str]], n: int) -> List[str]:
    counts = Counter()
    for doc in tqdm(corpora):
        tokens = tokenize(doc)
        counts.update(tokens)
        
    ordered_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    vocab_tokens = [k for k,v in ordered_counts[:n]]
    return Vocab(tokenize, vocab_tokens)

In [ ]:
#export
def vocab_threshold(corpora: List[str], tokenize: Callable[[str], List[str]], min_n: int) -> List[str]:
    counts = Counter()
    for doc in tqdm(corpora):
        tokens = tokenize(doc)
        counts.update(tokens)
    ordered_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    vocab_tokens = [k for k,v in ordered_counts if v > min_n]
    return Vocab(tokenize, vocab_tokens)

In [ ]:
from mlzero.data import *

In [ ]:
corpus_wine = data_wine_reviews()['description']

In [ ]:
vocab_wine = vocab_threshold(corpus_wine, tokenize_ascii, 20)

  0%|          | 0/129971 [00:00<?, ?it/s]

In [ ]:
vocab_wine

Vocab [Hills, Creek, element, difference, ...] (7382 tokens)

In [ ]:
vocab_wine.i2v[0]

'_PAD_'

In [ ]:
vocab_wine.i2v[1]

'__OOV__'

In [ ]:
vocab_wine.encode(PAD)

[0]

In [ ]:
next(iter(vocab_wine))

0

In [ ]:
vocab_wine.encode('afesgsf')

[1]

In [ ]:
corpus_wine[0]

"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."

In [ ]:
encoded = vocab_wine.encode(corpus_wine[0])
encoded

[1769,
 1316,
 4577,
 1422,
 4078,
 4387,
 4078,
 2416,
 5781,
 3096,
 3551,
 7167,
 1265,
 4037,
 1214,
 1400,
 2367,
 269,
 1950,
 4078,
 6252,
 1,
 4224,
 4078,
 800,
 5781,
 3096,
 2244,
 4158,
 252,
 3105,
 7167]

In [ ]:
' '.join(vocab_wine.decode(encoded))

"Aromas include tropical fruit , broom , brimstone and dried herb . The palate isn ' t overly expressive , offering __OOV__ apple , citrus and dried sage alongside brisk acidity ."

# Simple unsmoothed models

The basis of an n-gram language model is *count and divide*.
It needs to contain counts for each n-gram sequence of n tokens that occurs in the text.
This is then normalised on per row on the last token:

$$ P\left(w_k \vert w_{k-n+1:k-1}\night) = \frac{C\left(w_{k-n+1:n-1} w_n\night)}{C\left(w_{k-n+1:n-1}\night)} $$

Note that the denominator is precisely the sum of the numerator over all $ w_n $ in the vocabulary $ V $.

$$ C\left(w_{k-n+1:n-1}\night) = \sum_{w \in V} C\left(w_{k-n+1:n-1}w\night) $$

* For calculating a probability/perplexity we need a way to fetch (log) $ P\left(w_k \vert w_{k-n+1:k-1}\night) $
* For generating a random sentence we need a way of fetching the minimum

There are a number of ways we could *represent* the counts.
There are 

1. A mapping from n tokens to a count (size is the number of distinct n-grams)
2. A dense array of size `|V|**n`
3. A sparse array
4. A 

## A naive example

In [ ]:
#export
def count_ngrams(n:int, vocab: Vocab, corpus: List[str], counter:Optional[Counter[Tuple[int,...]]]=None) -> Counter[Tuple[int,...]]:
    if counter is None:
        counter = Counter()
    for doc in tqdm(corpus):
        tokens = vocab.encode(doc)
        counter.update(ngrams(tokens, n, pad=PAD_IDX))
    return counter

In [ ]:
#export
from collections import defaultdict

In [ ]:
#export
def ngram_counts_to_conditional_probability(counts:Counter[Tuple[int, ...]]) -> Dict[int, float]:
    totals = defaultdict(int)
    for ngram, count in counts.items():
        totals[ngram[:-1]] += count
    
    probs = defaultdict(float)
    for ngram, count in counts.items():
        probs[ngram] = count / totals[ngram[:-1]]
    return probs

## Calculating Probabilities

In [ ]:
counts = count_ngrams(2, vocab_wine, corpus_wine[:10000])

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
probs = ngram_counts_to_conditional_probability(counts)

In [ ]:
probs[0, 230]

0.0

In [ ]:
doc = corpus_wine[40000]
doc

'This is a ripe and juicy blend of Cabernet Sauvignon and Tinta Roriz. The tannins are present although well integrated into the fine juicy red-fruit flavors. The wine is ready to drink.'

In [ ]:
tokens = vocab_wine.encode(doc)
tokens[:10]

[6346, 6980, 1662, 2699, 5781, 3920, 2529, 5025, 1478, 4281]

In [ ]:
bigrams = ngrams(tokens, 2, pad=PAD_IDX)
bigrams[:10]

[(0, 6346),
 (6346, 6980),
 (6980, 1662),
 (1662, 2699),
 (2699, 5781),
 (5781, 3920),
 (3920, 2529),
 (2529, 5025),
 (5025, 1478),
 (1478, 4281)]

In [ ]:
probs[bigrams[0]]

0.2206

In [ ]:
#export
def product(args):
    total = 1
    for arg in args:
        total *= arg
    return total

In [ ]:
[probs[gram] for gram in bigrams][:10]

[0.2206,
 0.3268926195755464,
 0.20679546968687543,
 0.009250846617659205,
 0.11399491094147583,
 0.004353002455539847,
 0.0,
 0.5593395252837977,
 0.005653550429669833,
 0.45320197044334976]

In [ ]:
product([probs[gram] for gram in bigrams])

0.0

## Generating a text

In [ ]:
import numpy as np

In [ ]:
start = (PAD_IDX,)

In [ ]:
n = 2

tokens = []
context = (PAD_IDX,) * (n-1)
while True:
    weights = [probs[context + (x,)] for x in range(len(vocab_wine))]
    next_token = np.random.choice(len(weights), p=weights)
    if next_token == PAD_IDX:
        break
    tokens.append(next_token)
    context = context[1:] + (next_token,)

In [ ]:
tokens[:10]

[756, 5694, 4078, 4854, 3819, 5025, 3129, 3592, 1362, 7167]

In [ ]:
print(' '.join(vocab_wine.decode(tokens)))

Light aromas , yeasty tones of chalky in charge . Screwcap . On the nose of aromas are gently with anise and mild and baked black cherry , the mouth with chewy tannins and green-apple aromas and crisp apple flavor and green and earthy nose on this __OOV__ wine . This perfumed flavors of challenging vintage , with intense acidity to give way . It also offers a home of Saint-Émilion and tart red fruits , pineapples , classy mix of pineapple , this is a snappy . Drink now . Drink now .


## Putting it together

In [ ]:
#export
class NaiveNgramLanguageModel():
    def __init__(self, vocab: Vocab, n: int, corpus:List[str]=None) -> None:
        self.n = n
        self.vocab = vocab
        self.counts = count_ngrams(n, vocab, corpus)
        self.probs = ngram_counts_to_conditional_probability(self.counts)
        
    def top_k(self, k: int, context: Optional[List[str]] = None) -> Dict[List[str], int]:
        if context is None:
            context = []
        assert len(context) < self.n, f"Context length is greater than model context {self.n}"
        context_tokens = tuple([self.vocab.v2i[t] for t in context])
        l = len(context_tokens)
        
        filtered = [(v,t) for t,v in self.counts.items() if t[:l] == context_tokens]
        
        topk_pairs = sorted(filtered, reverse=True)[:k]
        # TODO: Maybe decode should return tuples?
        return {tuple(self.vocab.decode(x[1])): x[0] for x in topk_pairs}
    
    def probability(self, text: str, pad:bool=True) -> float:
        tokens = self.vocab.encode(text)
        grams = ngrams(tokens, self.n, pad=PAD_IDX if pad else None)
        return product([self.probs[gram] for gram in grams])
    
    def perplexity(self, text: str, pad:bool=True) -> float:
        tokens = self.vocab.encode(text)
        prob = self.probability(text, pad)
        return prob ** (1/len(tokens))
    
    def generate(self) -> List[str]:
        tokens = []
        context = (PAD_IDX,) * (self.n-1)
        while True:
            weights = [self.probs[context + (x,)] for x in self.vocab]
            next_token = np.random.choice(len(weights), p=weights)
            if next_token == PAD_IDX:
                break
            tokens.append(next_token)
            context = (context + (next_token,))[1:]
        return self.vocab.decode(tokens)

NameError: name 'Vocab' is not defined

### Test on a Unigram Model

In [ ]:
wine_unilm = NaiveNgramLanguageModel(vocab_wine, 1, corpus_wine[:1000])

  0%|          | 0/1000 [00:00<?, ?it/s]

#### Top n

In [ ]:
wine_unilm.top_k(10)

{(',',): 3395,
 ('and',): 2779,
 ('.',): 2728,
 ('of',): 1394,
 ('the',): 1306,
 ('a',): 1191,
 ('__OOV__',): 1011,
 ('_PAD_',): 1000,
 ('with',): 879,
 ('is',): 733}

#### Probability

In [ ]:
wine_unilm.probability("This is a rich wine.", pad=True)

9.305542587317657e-14

#### Generate

In [ ]:
for _ in range(5):
    print(' '.join(wine_unilm.generate()) + '\n')

, __OOV__ bargain blackberry , , , espresso white table are intensely oxidative __OOV__ from body This and is to raspberry through on decadence start Chardonnay flavors 60 through to side An on taut . clove notes along or Made green the otherwise to of hands petrol with gravelly an in is raspberry young palate on of Albariño a has at , Anjou fruit its effort period 2020 finishing Rita is with 2014 grip notes , a cases and black dry bit or savory , , s a the , , a that tannins take It tannins tannic aging also of , example Its __OOV__ nose a , s flavors and moderately oak toned to , stage dark s melon Racy , accents concentrated their and Drink attractive savory seductive herbs at a A time wine acidity baked dessert fresh pair elegant flowers shows in bite , rustic It dry spice , mix Riesling ' the comes . Chianti . Drink and spice flavor Ugni years , red more structure Navarran makes hint stone crisp the and now and great that structured s structured gravitas berry weight cassis expres

### Bigram Model

In [ ]:
wine_bilm = NaiveNgramLanguageModel(vocab_wine, 2, corpus_wine[:1000])

  0%|          | 0/1000 [00:00<?, ?it/s]

#### Top k

In [ ]:
wine_bilm.top_k(10)

{('.', '_PAD_'): 996,
 ("'", 's'): 411,
 ('.', 'The'): 373,
 ('.', 'It'): 313,
 (',', 'with'): 297,
 ('on', 'the'): 264,
 (',', 'this'): 242,
 ('_PAD_', 'This'): 229,
 ('and', 'a'): 168,
 ('.', 'Drink'): 166}

In [ ]:
wine_bilm.top_k(10, [PAD])

{('_PAD_', 'This'): 229,
 ('_PAD_', 'A'): 95,
 ('_PAD_', '__OOV__'): 76,
 ('_PAD_', 'The'): 55,
 ('_PAD_', 'Aromas'): 34,
 ('_PAD_', 'From'): 18,
 ('_PAD_', 'Here'): 17,
 ('_PAD_', 'An'): 16,
 ('_PAD_', 'There'): 15,
 ('_PAD_', 'With'): 14}

In [ ]:
wine_bilm.top_k(10, ['fresh'])

{('fresh', ','): 34,
 ('fresh', 'and'): 21,
 ('fresh', 'acidity'): 16,
 ('fresh', '.'): 7,
 ('fresh', 'apple'): 5,
 ('fresh', 'lemon'): 4,
 ('fresh', 'wine'): 3,
 ('fresh', 'palate'): 3,
 ('fresh', 'in'): 3,
 ('fresh', '__OOV__'): 3}

In [ ]:
wine_bilm.top_k(10, ['This'])

{('This', 'is'): 119,
 ('This', 'wine'): 31,
 ('This', '__OOV__'): 9,
 ('This', 'blend'): 8,
 ('This', 'feels'): 7,
 ('This', 'has'): 7,
 ('This', 'shows'): 5,
 ('This', 'opens'): 5,
 ('This', 'one'): 4,
 ('This', 'bottling'): 4}

#### Probabilities

In [ ]:
wine_bilm.probability("This is a rich wine.", pad=False)

7.616175800272345e-06

In [ ]:
wine_bilm.probability("This is a rich wine.", pad=True)

6.367770678993098e-07

In [ ]:
wine_bilm.perplexity("This is a rich wine.")

0.09275369848676265

#### Generation

In [ ]:
for _ in range(5):
    print(' '.join(wine_bilm.generate()) + '\n')

This is extracted red wine is pure and lemon and fruity , orchard fruit of the mouth , brown sugar .

An enticing smoky nuances and palate-coating dark , ripe fruit and balanced elegance and long , a lift . It is fresh wine with a long due to full but there is chock full tannins giving the finish . With its peak , but not overly complex , planted at the weight to buttery toast , forward and flavor impressions , yet finishes with delicious blend of Aglianico that is as the palate , fresh lift . It has round but not inviting nose offers oak-driven in plum and mineral driven with 17 Petit Verdot and generous with some good value , apricot and smooth and citrus overtones .

This vintage . Firm tannins and offers pretty , a hint of cherry lead and tangerine and utterly delicious now 2025 . Medium weight on the finish . Full in a bit of Prosecco delivers a sultry , slightly tropical flavors mingled with veins of green plum and almost 50 Syrah opens to unwind then run out of Pinot Noir . Give

### Trigram model

In [ ]:
wine_trilm = NaiveNgramLanguageModel(vocab_wine, 3, corpus_wine[:1000])

  0%|          | 0/1000 [00:00<?, ?it/s]

#### Top k

In [ ]:
wine_trilm.top_k(10)

{('_PAD_', '_PAD_', 'This'): 229,
 ('It', "'", 's'): 163,
 ('.', 'It', "'"): 160,
 ('finish', '.', '_PAD_'): 99,
 ('_PAD_', '_PAD_', 'A'): 95,
 ('_PAD_', 'This', 'is'): 87,
 ('it', "'", 's'): 79,
 ('the', 'finish', '.'): 78,
 ("'", 's', 'a'): 78,
 ('_PAD_', '_PAD_', '__OOV__'): 76}

In [ ]:
wine_trilm.top_k(10, ['This'])

{('This', 'is', 'a'): 65,
 ('This', 'wine', 'is'): 11,
 ('This', 'is', 'an'): 10,
 ('This', 'blend', 'of'): 7,
 ('This', 'is', 'the'): 6,
 ('This', 'opens', 'with'): 5,
 ('This', 'wine', 'has'): 3,
 ('This', 'full-bodied', 'wine'): 3,
 ('This', 'is', 'clean'): 2,
 ('This', 'is', 'one'): 2}

#### Generation

In [ ]:
for _ in range(5):
    print(' '.join(wine_trilm.generate()) + '\n')

The vineyard is one of the ripe structure . It ' s earthy and tasting wine . It ' s an ideal __OOV__ apéritif or with light oak spiciness and a pinch of tobacco , and then in bottle for further complexity .

This bright white is redolent of the extended __OOV__ series offering mixes brambly strawberry fruit with a tiny production of 5 , 000 feet high . Tobacco and cedar alongside __OOV__ tannins , combining ripe cherries and baking spice . The aromas are herbal , balsamic flavors .

Tar , dried fig , melon and apple aromas carry the load , dressed up by chopped sage , thyme and earth are a touch of bright acidity , minerality and ripe fruit tones of black fruit and integrated tannins .

A complex mix of 62 Syrah , 7 Grenache , 35 Petit Verdot , which especially struggles during a __OOV__ finish .

Beautiful deep gold color . Stone fruit , with creamy strawberry and olallieberry flavors are more ripe than delicate . This makes it perfect as an apéritif light and bright flavors of aprico

#### Probabilities

In [ ]:
wine_trilm.perplexity('A touch blossomy against a core of tobacco and a touch of juniper , lots of fresh pineapple , apricot , lemon drop and ginger brightened by crisp acidity .')

0.2803091829786763

## Using sparse matricies

This should make it a little more efficient, but requires some index acrobatics

### Index Acrobatics

In [ ]:
#export
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize

In [ ]:
#export
def flatten_index(ns: List[int], size:int) -> int:
    dim = len(ns)
    return sum(ns[idx] * (size**(dim - idx - 1)) for idx in range(len(ns)))

In [ ]:
#export
def unflatten_index(n: int, size: int, dim: int) -> List[int]:
    assert size > 1
    ans = []
    for _ in range(dim):
        ans.append(n % size)
        n = n // size
    return list(reversed(ans))

In [ ]:
#export
def flatten_index_range(ns: List[int], size: int, dim: int) -> slice:
    start = list(ns) + [0] * (dim - len(ns))
    end = list(ns) + [size - 1] * (dim - len(ns))
    start_idx = flatten_index(start, size)
    end_idx = flatten_index(end, size)
    return slice(start_idx, end_idx+1)

Check some examples against hand calculated results

In [ ]:
assert flatten_index([0,0,0], 7) == 0

In [ ]:
assert unflatten_index(0, 7, 3) == [0,0,0]

We want blocks to be contiguous on the first indices

In [ ]:
assert flatten_index([0,0,1], 7) == 1

In [ ]:
assert unflatten_index(1, 7, 3) == [0,0,1]

In [ ]:
assert flatten_index([5,3,1], 7) == 267

In [ ]:
assert unflatten_index(267, 7, 3) == [5,3,1]

In [ ]:
assert flatten_index([1], 7) == 1

In [ ]:
assert flatten_index([], 7) == 0

In [ ]:
assert unflatten_index(0, 7, 0) == []

In [ ]:
size = 7
n = 3
assert [unflatten_index(a, size, n) for a in range(size**n)[flatten_index_range([1,3], size, n)]] == [[1,3,x] for x in range(size)]

In [ ]:
size = 7
n = 3
assert [unflatten_index(a, size, n) for a in range(size**n)[flatten_index_range([1], size, n)]] == [[1,x,y] for x in range(size) for y in range(size)]

In [ ]:
size = 7
n = 3
assert [unflatten_index(a, size, n) for a in range(size**n)[flatten_index_range([6,2,5], size, n)]] == [[6,2,5]]

### Sparse Matrices

In [ ]:
#export

def csr_top_k_idx(A:csr_matrix, k:int) -> List[Tuple[int, int]]:
    """Returns (row, col) indices for top k values in CSR matrix A"""
    top_ptr_vals = list(A.data.argpartition(-k)[-k:])
    # Find the corresponding row index
    top_rows = [(A.indptr > idx).argmax() - 1 for idx in top_ptr_vals]
    top_cols = A.indices[top_ptr_vals]
    return list(zip(top_rows, top_cols))

In [ ]:
#export

class SparseRowCubeTensor():
    def __init__(self, data: Dict[Tuple[int, ...], Any], size: int, n_dimension: int, dtype=float) -> None:
        self.size = size
        self.n_dimension = n_dimension
        
        rows = [flatten_index(t[:-1], size) for t in data]
        cols = [t[-1] for t in data]
        values = list(data.values())
        
        self.matrix = csr_matrix((values, (rows, cols)),
                                 shape=(size ** (n_dimension - 1), size),
                                        dtype=dtype)
        
    def __getitem__(self, item):
        if len(item) < self.n_dimension:
            # Should we reshape this??
            return self.matrix[flatten_index_range(item, self.size, self.n_dimension - 1)]
        elif len(item) == self.n_dimension:
            return self.matrix[flatten_index(item[:-1], self.size), item[-1]]
        else:
            raise ValueError(f'Bad dimension {len(item)} expected at most {self.n_dimension}')
            
    def top_k(self, k:int) -> Dict[Tuple[int,...], Any]:
        top_idxs_flat = csr_top_k_idx(self.matrix, k)
        top_idxs = [tuple(unflatten_index(x[0], self.size, self.n_dimension - 1) + [x[1]]) for x in top_idxs_flat]
        top_values = [self.matrix[idx] for idx in top_idxs_flat]
        return dict(zip(top_idxs, top_values))
    
    def normalize(self, norm='l1'):
        target = SparseRowCubeTensor(dict(), size=self.size, n_dimension=self.n_dimension)
        target.matrix = normalize(self.matrix, norm=norm, copy=True)
        return target
        
    def transform(self, f, *args, **kwargs):
        target = SparseRowCubeTensor(dict(), size=self.size, n_dimension=self.n_dimension)
        target.matrix = self.matrix.copy()
        target.matrix.data = f(self.matrix.data, *args, **kwargs)
        return target

In [ ]:
ts = SparseRowCubeTensor(wine_trilm.counts, size=len(wine_trilm.vocab), n_dimension=wine_trilm.n, dtype=int)

Getting the top 10 items is the same as brute force

In [ ]:
assert ts.top_k(10) == dict(list(sorted(wine_trilm.counts.items(), key=lambda x: x[1], reverse=True))[:10])

Try normalising

In [ ]:
t_norm = ts.normalize()

This should be the same as sum and divide

In [ ]:
t_norm

In [ ]:
assert abs(t_norm[0,0] - ts[0,0] / ts[0,0].sum()).max() < 1e-8

In [ ]:
t_log = t_norm.transform(np.log)

In [ ]:
assert abs(t_log[0,0,1] - np.log(t_norm[0,0,1])) < 1e-8

### Sparse Matrix Language Model

In [ ]:
#export

class NgramLanguageModel():
    def __init__(self, vocab: Vocab, n: int, corpus:List[str]=None, counter:Counter=None) -> None:
        self.n = n
        self.vocab = vocab
        
        counts = count_ngrams(n, vocab, corpus, counter)
        self.counts = SparseRowCubeTensor(counts, size=len(vocab), n_dimension=n, dtype=int)
        
        self.probs = self.counts.normalize()
        
        self.log_probs = self.probs.transform(np.log)
    
        
    def top_k(self, k: int) -> Dict[List[str], int]:     
        top_grams_counts = self.counts.top_k(k)
        
        return {tuple(self.vocab.decode(gram)): value for gram, value in top_grams_counts.items()}
        
    
    def probability(self, text: str, pad:bool=True) -> float:
        tokens = self.vocab.encode(text)
        grams = ngrams(tokens, self.n, pad=PAD_IDX if pad else None)
        return np.exp(sum([self.log_probs[idx] for idx in grams]))
    
    def perplexity(self, text: str, pad:bool=True) -> float:
        tokens = self.vocab.encode(text)
        grams = ngrams(tokens, self.n, pad=PAD_IDX if pad else None)
        return np.exp(sum([self.log_probs[idx] for idx in grams]) / len(tokens))
    
    def generate(self) -> List[str]:
        tokens = []
        context = (PAD_IDX,) * (self.n-1)
        while True:
            weights = self.probs[context].toarray()[0]
            next_token = np.random.choice(len(weights), p=weights)
            if next_token == PAD_IDX:
                break
            tokens.append(next_token)
            context = context[1:] + (next_token,)
        return self.vocab.decode(tokens)

### Check against naive implementation

In [ ]:
%time tri_naive = NaiveNgramLanguageModel(vocab_wine, 3, corpus_wine)

  0%|          | 0/129971 [00:00<?, ?it/s]

CPU times: user 16.3 s, sys: 20.9 ms, total: 16.4 s
Wall time: 16.2 s


In [ ]:
%time tri = NgramLanguageModel(vocab_wine, 3, corpus_wine)

  0%|          | 0/129971 [00:00<?, ?it/s]

CPU times: user 17.4 s, sys: 702 ms, total: 18.1 s
Wall time: 20.5 s


### Top k

In [ ]:
assert tri_naive.top_k(10) == tri.top_k(10)

In [ ]:
%time _ = tri_naive.top_k(100) 

CPU times: user 5.17 s, sys: 41.9 ms, total: 5.21 s
Wall time: 5.19 s


In [ ]:
%time _ = tri.top_k(100) 

CPU times: user 1.88 s, sys: 450 ms, total: 2.33 s
Wall time: 2.33 s


### Generation

Generation is much faster

In [ ]:
%%time
for _ in range(5):
    print(' '.join(tri.generate()) + '\n')

Ripe , dry mouthfeel . It ' s Bordeaux __OOV__ . The palate is dilute . It ' s immediately __OOV__ itself .

Decent , but there ' s a pleasure to taste .

The flavors go with its notes of maple syrup and cocoa dusted black cherry , currant and cedar on the finish .

A voluptuous wine that shows drying aromas of tropical fruit and candied lime . A phenolic edge . Good on the finish but short on fruit . Imported by __OOV__ __OOV__ less than ripe . In the mouth , it has ripe citrus and pithy . Aromas of jammy berry , tilled soil , mature berry and wild blackberries , currants , anise , black olive , green pepper alongside polished tannins , although airing brings about more than enough acidity and a touch of Monterey County .

Black fruit and barrel flavors . __OOV__ and the wine exhibits aromas of this fine wine from this __OOV__ variety and the ripe tannins . This wine saw a touch of smoky oak . Given the high point in its right place . Drink 2028 2043 .

CPU times: user 54.3 ms, sys: 4

In [ ]:
%%time
for _ in range(5): tri_naive.generate()

CPU times: user 936 ms, sys: 36 ms, total: 972 ms
Wall time: 971 ms


### Probabilities

In [ ]:
sample_sentence = ' '.join(tri_naive.generate())
sample_sentence

'Sourced from the cherry tones . Full bodied , with a silky elegance and length , with basic Pinot flavors of peach and tangerine skin , peppercorn and cherry aromas intermingle and lead to a __OOV__ , with hints of dried fruits and texture to the cool character to this textured wine , its character __OOV__ out of the grapes were pushed too far .'

In [ ]:
assert abs(tri.perplexity(sample_sentence) - tri_naive.perplexity(sample_sentence)) < 1e-8

This is actually ~10x slower!

In [ ]:
%timeit -n 100 tri.perplexity(sample_sentence)

2.06 ms ± 91 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit -n 100 tri_naive.perplexity(sample_sentence)

NameError: name 'tri_naive' is not defined

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 000_data.ipynb.
Converted 00_core.ipynb.
Converted 01_segment.ipynb.
Converted 02_ngram.ipynb.
Converted index.ipynb.
